In [ ]:
import requests
import json
import pandas as pd
from sklearn.model_selection import train_test_split
import re

# Get dishes

In [ ]:
get_dishes_url='https://gappapi.deliverynow.vn/api/v5/buyer/store/dishes'

In [ ]:
header={
    'accept': 'application/json, text/plain, */*'
    ,'x-foody-client-id': 'fa42efc4-7303-4cd8-b71f-93359fb6d50a'
    ,'x-foody-client-type': '4'
    ,'x-foody-app-type': '1004'
    ,'x-foody-client-version': '5.23.0'
    ,'x-foody-api-version': '1'
    ,'x-foody-client-language': 'en'
    ,'x-foody-access-token': 'c1d1af077e1f81a093e156172fd128d21e63f7eaf343922c7d5036c5f645de7aa08462fae200029d712a47c68ea43ac619da5a0c07421aab722e2e9be95b90db'
    #Có vẻ token không đổi, có thể chỉ dựa vào restaurant_id
    ,'user-agent': 'NOW/5.13.2 (sdk_gphone64_x86_64; android 33; Scale/2.75)'
    ,'x-foody-client-rn-version': '5.13.2'
    ,'x-foody-device-fingerprint': None
    ,'accept-encoding': 'gzip'
}

In [ ]:
query={
    'restaurant_id': 1137988
}

In [ ]:
respone=requests.get(get_dishes_url, params=query, headers=header)

In [ ]:
df=json.loads(respone.text)

In [ ]:
def get_dishes(url, query, header,restaurant_list):
    catalog_id=[]
    catalog_name=[]
    catalog_rank=[]
    catalog_partner_catalog_id=[]
    catalog_description=[]
    dish_restaurant_id=[]
    dish_id=[]
    dish_name=[]
    dish_partner_dish_id=[]
    dish_listing_status=[]
    dish_description=[]
    dish_total_like=[]
    dish_rank=[]
    dish_picture_label=[]
    dish_is_hidden=[]
    dish_price=[]
    dish_is_group_discount_item=[]
    dishes_property_info=[]
    dish_total_order = []
#     temp=0
    for restaurant_id in restaurant_list:
#         temp = temp+1
#         if temp == 100:
#             break
        query['restaurant_id']=restaurant_id
        respone=json.loads(requests.get(get_dishes_url, params=query, headers=header).text)
        if 'data' in respone:
            if 'catalogs' in respone['data']:
                for catalog in respone['data']['catalogs']:
                    for dishes in catalog['dishes']:
                        catalog_id.append(catalog['id'])
                        catalog_name.append(catalog['name'])
                        catalog_rank.append(catalog['rank'])
                        if 'partner_catalog_id' in catalog:
                            catalog_partner_catalog_id.append(catalog['partner_catalog_id'])
                        else:
                            catalog_partner_catalog_id.append(None)
                        if 'description' in catalog:
                            catalog_description.append(catalog['description'])
                        else:
                            catalog_description.append(None)
                        dish_restaurant_id.append(dishes['restaurant_id'])
                        dish_id.append(dishes['id'])
                        dish_name.append(dishes['name'])
                        dish_partner_dish_id.append(dishes['partner_dish_id'])
                        dish_listing_status.append(dishes['listing_status'])
                        dish_description.append(dishes['description'])
                        dish_total_like.append(dishes['total_like'])
                        dish_rank.append(dishes['rank'])
                        if 'picture_label' in dishes:
                            dish_picture_label.append(dishes['picture_label'])
                        else:
                            dish_picture_label.append(None)
                        dish_is_hidden.append(dishes['is_hidden'])
                        dish_price.append(dishes['price'])
                        dish_total_order.append(dishes['total_order'])
                        dish_is_group_discount_item.append(dishes['is_group_discount_item'])
                        dishes_property_info.append(json.dumps(dishes['property_info']))
            else:
                continue

    data_dict={
        'catalog_id':catalog_id
        ,'dish_total_order':dish_total_order
        ,'catalog_name':catalog_name
        ,'catalog_rank':catalog_rank
        ,'catalog_partner_catalog_id':catalog_partner_catalog_id
        ,'catalog_description':catalog_description
        ,'dish_restaurant_id':dish_restaurant_id
        ,'dish_id':dish_id
        ,'dish_name':dish_name
        ,'dish_partner_dish_id':dish_partner_dish_id
        ,'dish_listing_status':dish_listing_status
        ,'dish_description':dish_description
        ,'dish_total_like':dish_total_like
        ,'dish_rank':dish_rank
        ,'dish_picture_label':dish_picture_label
        ,'dish_is_hidden':dish_is_hidden
        ,'dish_price':dish_price
        ,'dish_is_group_discount_item':dish_is_group_discount_item
        ,'dishes_property_info':dishes_property_info
  }
    return pd.DataFrame(data_dict)

In [ ]:
def runn(inputt, outputt):
    district=pd.read_excel(inputt)
    list_restaurant_id=list(district['restaurant_id'])
    dishes_df=get_dishes(get_dishes_url, query, header,list_restaurant_id)
    check = len(dishes_df)
    print("len(dishes_df) ", check)
    if check < 107000:
        dishes_df.to_excel(outputt)
        print(outputt, "okela")
    else:
        district_name,b =re.split('\.',outputt)
        if check < 210000:
            dishes_df_1, dishes_df_2 = train_test_split(dishes_df, test_size=0.5  , shuffle=False)
            print("2v",len(dishes_df_1),len(dishes_df_2))
            dishes_df_1.to_excel(district_name + "_1." + b)
            print(district_name + "_1." + b, "okela")
            dishes_df_2.to_excel(district_name + "_2." + b)
            print(district_name + "_2." + b, "okela")
        else:
            dishes_df_, dishes_df_3 = train_test_split(dishes_df, test_size=0.33  , shuffle=False)
            dishes_df_1, dishes_df_2 = train_test_split(dishes_df_, test_size=0.5  , shuffle=False)
            print("3ver",len(dishes_df_1),len(dishes_df_2),len(dishes_df_3))
            dishes_df_1.to_excel(district_name + "_1." + b)
            print(district_name + "_1." + b, "okela")
            dishes_df_2.to_excel(district_name + "_2." + b)
            print(district_name + "_2." + b, "okela")
            dishes_df_3.to_excel(district_name + "_3." + b)
            print(district_name + "_3." + b, "okela")

In [ ]:
mypath = "../input/ptdltm/Nam_res_id/retaurant_id/"
runn(mypath + 'restaurant_id_Quận 2.xlsx','dishes_Quận 2.xlsx') ##ok
runn(mypath + 'restaurant_id_Quận 1.xlsx','dishes_Quận 1.xlsx')
runn(mypath + 'restaurant_id_Quận 9.xlsx','dishes_Quận 9.xlsx')
runn(mypath + 'restaurant_id_Bình Thạnh.xlsx','dishes_Bình Thạnh.xlsx')

In [ ]:
runn(mypath + 'restaurant_id_Nhà Bè.xlsx','dishes_Nhà Bè.xlsx')##ok
runn(mypath + 'restaurant_id_Cần Giờ.xlsx','dishes_Cần Giờ.xlsx')##ok
runn(mypath + 'restaurant_id_Củ Chi.xlsx','dishes_Củ Chi.xlsx') ##ok
runn(mypath + 'restaurant_id_Tân Phú.xlsx','dishes_Tân Phú.xlsx')
runn(mypath + 'restaurant_id_Thủ Đức.xlsx','dishes_Thủ Đức.xlsx')

In [ ]:
runn(mypath + 'restaurant_id_Bình Tân.xlsx','dishes_Bình Tân.xlsx') ##ok
runn(mypath + 'restaurant_id_Hóc Môn.xlsx','dishes_Hóc Môn.xlsx') ##ok
runn(mypath + 'restaurant_id_Gò Vấp.xlsx','dishes_Gò Vấp.xlsx')


In [ ]:
runn(mypath + 'restaurant_id_Tân Bình.xlsx','dishes_Tân Bình.xlsx')
runn(mypath + 'restaurant_id_Quận 7.xlsx','dishes_Quận 7.xlsx')
runn(mypath + 'restaurant_id_Bình Chánh.xlsx','dishes_Bình Chánh.xlsx')

In [ ]:
runn(mypath + 'restaurant_id_Quận 3.xlsx','dishes_Quận 3.xlsx') ##ok
runn(mypath + 'restaurant_id_Quận 10.xlsx','dishes_Quận 10.xlsx')
runn(mypath + 'restaurant_id_Quận 8.xlsx','dishes_Quận 8.xlsx')
runn(mypath + 'restaurant_id_Quận 5.xlsx','dishes_Quận 5.xlsx')

In [ ]:
runn(mypath + 'restaurant_id_Quận 4.xlsx','dishes_Quận 4.xlsx') ##ok
runn(mypath + 'restaurant_id_Quận 6.xlsx','dishes_Quận 6.xlsx')##ok
runn(mypath + 'restaurant_id_Quận 11.xlsx','dishes_Quận 11.xlsx')##ok
runn(mypath + 'restaurant_id_Quận 12.xlsx','dishes_Quận 12.xlsx')##ok
runn(mypath + 'restaurant_id_Phú Nhuận.xlsx','dishes_Phú Nhuận.xlsx')##ok